In [1]:
import json, csv
import pandas as pd

In [2]:
with open("output/enhanced_dataset.json", 'r') as fp:
    dataset = json.load(fp)

In [3]:
admin = pd.read_csv('resources/admin_areas.csv',index_col='uniq_id')

In [4]:
cyrillic_translit={u'\u0410': 'A', u'\u0430': 'a',u'\u0411': 'B', u'\u0431': 'b',u'\u0412': 'V', u'\u0432': 'v',u'\u0413': 'G', u'\u0433': 'g',u'\u0414': 'D', u'\u0434': 'd',u'\u0415': 'E', u'\u0435': 'e',u'\u0416': 'Zh', u'\u0436': 'zh',u'\u0417': 'Z', u'\u0437': 'z',u'\u0418': 'I', u'\u0438': 'i',u'\u0419': 'I', u'\u0439': 'i',u'\u041a': 'K', u'\u043a': 'k',u'\u041b': 'L', u'\u043b': 'l',u'\u041c': 'M', u'\u043c': 'm',u'\u041d': 'N', u'\u043d': 'n',u'\u041e': 'O', u'\u043e': 'o',u'\u041f': 'P', u'\u043f': 'p',u'\u0420': 'R', u'\u0440': 'r',u'\u0421': 'S', u'\u0441': 's',u'\u0422': 'T', u'\u0442': 't',u'\u0423': 'U', u'\u0443': 'u',u'\u0424': 'F', u'\u0444': 'f',u'\u0425': 'Kh', u'\u0445': 'kh',u'\u0426': 'Ts', u'\u0446': 'ts',u'\u0427': 'Ch', u'\u0447': 'ch',u'\u0428': 'Sh', u'\u0448': 'sh',u'\u0429': 'Shch', u'\u0449': 'shch',u'\u042a': '"', u'\u044a': '"',u'\u042b': 'Y', u'\u044b': 'y',u'\u042c': "'", u'\u044c': "'",u'\u042d': 'E', u'\u044d': 'e',u'\u042e': 'Iu', u'\u044e': 'iu',u'\u042f': 'Ia', u'\u044f': 'ia',u'\u0462': 'E', u'\u0463': 'e'}

def transliterate(word, translit_table):
    """
    Transliterates 'word' based on the key/value pairs in 'translit_table'
    """
    if type(word)!=str:
        return None
    else:
        converted_word = ''
        for char in word:
            transchar = ''
            if char in translit_table:
                transchar = translit_table[char]
            else:
                transchar = char
            converted_word += transchar
        return converted_word

In [13]:
dataset["528"]

{'SOURCE': 'http://babel.hathitrust.org/cgi/pt?id=uc1.b4583201;view=1up;seq=87',
 'admin1_new_orth': 'Екатеринославского Наместничества',
 'admin1_old_orth': 'Екатеринославскаго Намѣстничества',
 'admin1_partofID': 8.0,
 'admin1_std_name': 'Екатеринославское Наместничество',
 'admin1_stem': 'екатеринославск',
 'admin2_new_orth': None,
 'admin2_old_orth': None,
 'admin2_partofID': None,
 'admin2_stem': 'not_in_text',
 'admin_hierarchy': 'екатеринославск|not_in_text',
 'alt_name_new_orth': None,
 'alt_name_old_orth': None,
 'desc_machine_translated': 'PETROVSKAYA fortress Yekaterinoslav region. In the case thereof, in the year six fairs.',
 'description_new_orth': 'ПЕТРОВСКАЯ , крепость Екатеринославского Наместничества. В оной бывает в году шесть ярмарок.',
 'description_old_orth': 'ПЕТРОВСКАЯ , крѣпость Екатеринославскаго Намѣстничества. Въ оной бываетъ въ году шесть ярмарокъ.',
 'featuretype_new_orth': 'крепость',
 'featuretype_new_orth_lc': "krepost'",
 'featuretype_old_orth': 'крѣпо

In [6]:
sourceLink = "https://www.zotero.org/jaguillette/items/collectionKey/3KQ5I42Q/itemKey/5AI864SV"
startDate = 1785
endDate = 1796
confidence = "confident"

In [10]:
def add_admin_to_places(adminID,adminType,writer=place_csv_writer,admin_df=admin):
    name = admin_df.loc[int(adminID)]['lc_translit_new']
    lat = admin_df.loc[int(adminID)]['y_coord']
    lon = admin_df.loc[int(adminID)]['x_coord']
    adminID = int(adminID)+805
    place_csv_writer.writerow([adminID,name,lat,lon,'seat',adminType,0])
    return

In [14]:
with open("graph/(places).csv","w+") as place_fp, open("graph/[partof].csv","w+") as partof_fp:
    # Setup place csv writer and headers
    place_csv_writer = csv.writer(place_fp)
    place_csv_writer.writerow(['ID','title','reprLat','reprLong','reprType','geonamesType','provisional'])
    # Setup partof csv writer and headers
    partof_csv_writer = csv.writer(partof_fp)
    partof_csv_writer.writerow(['ID','child','parent','sourceLink','startDate','endDate','confidence'])
    def add_admin_to_places(adminID,adminType,writer=place_csv_writer,admin_df=admin):
        name = admin_df.loc[int(adminID)]['lc_translit_new']
        lat = admin_df.loc[int(adminID)]['y_coord']
        lon = admin_df.loc[int(adminID)]['x_coord']
        adminID = int(adminID)+805
        place_csv_writer.writerow([adminID,name,lat,lon,'seat',adminType,0])
        return
    # keep track of admin places entered
    partof_places_entered = []
    for k,v in dataset.items():
        if 'admin2_partofID' in v and v['admin2_partofID']:
            admin2_id = int(v['admin2_partofID']) + 805
            # Check if admin2 recorded, add if not
            if v['admin2_partofID'] not in partof_places_entered:
                try:
                    add_admin_to_places(v['admin2_partofID'],'ADM2')
                except KeyError:
                    print("KeyError! ({})".format(v['admin2_partofID']))
                    print(v)
                    print()
                partof_places_entered.append(v['admin2_partofID'])
            # Add partof relationship  place -> admin2
            partof_csv_writer.writerow([None,k,admin2_id,sourceLink,startDate,endDate,confidence])
            if 'admin1_partofID' in v and v['admin1_partofID']:
                admin1_id = int(v['admin1_partofID']) + 805
                # Check if admin1 recorded, add if not
                if v['admin1_partofID'] not in partof_places_entered:
                    add_admin_to_places(v['admin1_partofID'],'ADM1')
                    partof_places_entered.append(v['admin1_partofID'])
                # Add partof relationship place -> admin1 and admin2 -> admin1
                partof_csv_writer.writerow([None,k,admin1_id,sourceLink,startDate,endDate,confidence])
                partof_csv_writer.writerow([None,admin2_id,admin1_id,sourceLink,startDate,endDate,confidence])
            else:
                print("The following has an admin2 ID, but no admin1 ID:")
                print(v['description_new_orth']) # make sure that's right, since it's weird
                print(v['desc_machine_translated'])
                print()
        else:
            if 'admin1_partofID' in v and v['admin1_partofID']:
                admin1_id = int(v['admin1_partofID']) + 805
                # Check if admin1 recorded, add if not
                if v['admin1_partofID'] not in partof_places_entered:
                    add_admin_to_places(v['admin1_partofID'],'ADM1')
                    partof_places_entered.append(v['admin1_partofID'])
                # Add partof relationship place -> admin1
                partof_csv_writer.writerow([None,k,admin1_id,sourceLink,startDate,endDate,confidence])
            else:
                print("The following has no admin partof IDs:")
                print(v['description_new_orth']) # make sure that's right, since it's weird
                print(v['desc_machine_translated'])
                print()
        # Add place to Places
        if 'geo' in v:
            lon = v['geo']['x_coord']
            lat = v['geo']['y_coord']
            reprType = 'GeoNames'
        else:
            lon = None
            lat = None
            reprType = None
        place_csv_writer.writerow([k,v['name_new_orth_lc'],lat,lon,reprType,'PPL',0])

The following has no admin partof IDs:
МИТЯКИНСКАЯ , станица Донских Козаков. В ней всякой год в десятую пятницу по Пасхе бывает ярмарка, на которую пригоняют скот , также привозят вялую рыбу , холст, шелковые материи, сукна, деготь и горячие напитки.
MITYAKINSKAYA, village Don Cossacks. In it every year on the tenth Friday after Easter is fair, where cattle; also bring sluggish fish, canvas, silk fabrics, cloth, tar and hot drinks.

The following has no admin partof IDs:
ОЛИШЕВКА , местечко в округе города Киева. Здесь бывает в году две ярмарки ; на оных скупают приезжие купцы кожи , овчины , шубы , соль и прочее.
OLISHEVKA, town in the district of Kiev. Here is the year two fairs; buy add-on visiting merchants leather, sheepskin, fur coats, salt and so on.

The following has no admin partof IDs:
ЧЕРЕНЧИЦКОЙ, погост в уезде города Старой Рузы. Здесь бывает одна годовая ярмарка в день Ильинской пятницы, и купцы приезжают из Старой Рузы с мелочными товарами.
CHERENCHITSKOY, churchyard i